In [1]:
import os
import transformers

In [2]:
from transformers import BertTokenizer, BertModel

In [4]:
#加载预训练的、专用于bert的分词模型
tokenizer = BertTokenizer.from_pretrained('bert-base-chinese',trust_remote_code=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/110k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/269k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/624 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [5]:
# 加载预训练的bert
model = BertModel.from_pretrained('bert-base-chinese')

model.safetensors:   0%|          | 0.00/412M [00:00<?, ?B/s]

In [8]:
text = '虽然今天下雨，但我拿到了心仪的offer，因此非常开心！'

#将text输入分词器和编码模型
encoded_input = tokenizer(text,return_tensors='pt')

#将编码好的文字输入给预训练号的bert
output = model(**encoded_input)

In [9]:
output

BaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=tensor([[[ 1.1433, -0.0609,  1.7443,  ..., -0.5673,  0.6769, -0.5895],
         [ 0.0999,  1.0394,  1.3015,  ..., -0.2681, -0.0315,  0.1212],
         [-0.0700, -0.2058,  0.2773,  ...,  0.5447,  0.5957,  0.3067],
         ...,
         [ 1.9024, -0.2324,  0.9153,  ..., -0.1779,  0.6744, -0.0338],
         [ 0.7510, -0.3802,  1.7366,  ..., -0.8836,  0.7272, -0.4658],
         [ 0.7808, -0.3259,  1.2399,  ..., -0.7098,  0.7136, -0.6045]]],
       grad_fn=<NativeLayerNormBackward0>), pooler_output=tensor([[ 0.9999,  0.9999,  0.9996,  0.9359,  0.9717, -0.5743, -0.9979,  0.8505,
          0.9978, -0.9995,  1.0000,  0.9999,  0.9692, -0.9830,  0.9996, -0.9999,
         -0.9957,  0.9959,  0.7777,  0.3605,  0.9999, -1.0000, -0.8953, -0.6188,
         -0.7018,  0.9997,  0.9472, -0.8435, -1.0000,  0.9991,  0.9481,  0.9996,
          0.9966, -1.0000, -1.0000,  0.9917, -0.7756,  0.9961,  0.0712, -0.7999,
         -0.9981, -0.9976,  0.81

In [ ]:
# 自己定义如何使用bert输出的结果

In [10]:
import torch
from transformers import BertTokenizer, BertModel
import torch.nn as nn

# 加载BERT模型和分词器
tokenizer = BertTokenizer.from_pretrained('bert-base-chinese')
model = BertModel.from_pretrained('bert-base-chinese')



/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [12]:
#定义句子分类器
class BertSentenceClassifier(nn.Module):
  def __init__(self,bert_model,num_classes):
    super(BertSentenceClassifier,self).__init__()
    self.bert = bert_model
    self.classifier = nn.Linear(bert_model.config.hidden_size,num_classes)

  def forward(self, input_ids, attention_mask):
    outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
    pooler_output = outputs.pooler_output
    logits = self.classifier(pooler_output)
    return logits



In [26]:
text = "虽然今天下雨，但我拿到了心仪的offer，因此非常开心！"

#将文本转化成bert的输入格式
encoded_input = tokenizer(text, return_tensors = 'pt')

#初始化分类器，假设我们有两个分类标签，如消极和积极
classifier = BertSentenceClassifier(model,num_classes=2)

#获取分类结果
logits = classifier(encoded_input['input_ids'],encoded_input['attention_mask'])
probabilities = torch.softmax(logits, dim=1)

#打印分类结果
print("Logits:",logits)
print("Probabilities:",probabilities)

# 类别预测
predicted_class = torch.argmax(probabilities, dim=1).item()

# 判断句子的情感
if predicted_class == 0:
    sentiment = "Negative"
else:
    sentiment = "Positive"

print(f"预测的情感: {sentiment}")

Logits: tensor([[-0.1470,  0.6081]], grad_fn=<AddmmBackward0>)
Probabilities: tensor([[0.3197, 0.6803]], grad_fn=<SoftmaxBackward0>)
预测的情感: Positive
